In [ ]:
import pandas as pd
import numpy as np
from google.colab import drive
import requests
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler
import plotly.express as px
from wordcloud import WordCloud
from collections import Counter
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### 주식 보조 지표를 이용한 '재무적 기업 가치' 점수 책정

### 1) 종목 정보를 통해 기업 가치 지표 크롤링하기

동일 업종 PER은 NAVER에서 크롤링했으며 PER, ROE, EPS, EV/EBITDA는 fnguide에서 크롤링했음

In [ ]:
kospi = pd.read_csv('/content/drive/MyDrive/미래에셋/kospi/KOSPI_시가총액_상위500.csv', index_col = 0)
del kospi['업종']
kospi.head()

,종목코드,종목명,종가,대비,등락률,시가,고가,저가,거래량,거래대금,시가총액,상장주식수,중분류,소분류
0,005930,삼성전자,75300.0,0.0,0.00,75300.0,75600.0,74800.0,10103212.0,7.600385e+11,4.495246e+14,5.969783e+09,반도체와반도체장비,반도체와반도체장비
1,000660,SK하이닉스,105000.0,2000.0,1.94,103500.0,105500.0,103000.0,1999369.0,2.087660e+11,7.644025e+13,7.280024e+08,반도체와반도체장비,반도체와반도체장비
2,035420,NAVER,410000.0,11000.0,2.76,398000.0,416500.0,398000.0,1326870.0,5.432771e+11,6.734799e+13,1.642634e+08,미디어,양방향미디어와서비스
3,207940,삼성바이오로직스,925000.0,12000.0,1.31,919000.0,935000.0,919000.0,59758.0,5.532594e+10,6.120262e+13,6.616500e+07,제약과생물공학,제약
4,005935,삼성전자우,70700.0,-1000.0,-1.39,71000.0,71300.0,70700.0,1169578.0,8.295213e+10,5.817809e+13,8.228867e+08,반도체와반도체장비,반도체와반도체장비


In [ ]:
#시가총액 1위부터 100위까지 시가총액, 상장주식수, PER, PBR, EPS 정보 가져오기
stock_pointer = []
for code in list(kospi['종목코드'])[:100]:
  url = 'https://finance.naver.com/item/main.naver?code={}'.format(code)
  res = requests.get(url)
  soup = BeautifulSoup(res.text, 'lxml')
  info_list = soup.find('table', attrs={'class':'per_table'})
  #동일업종 PER 정보 가져오기
  try:
    upjong_PER = float(soup.find_all('div', attrs={'class':'gray'})[1].find('em').text)
  except:
    upjong_PER = None
  #PBR 정보 가져오기
  info_list = soup.find('table', attrs={'class':'per_table'})
  if info_list == None:
    PBR = None
  else:
    info_list = info_list.find_all('tr')
    PBR = info_list[2].find('em').text #PBR
    if PBR == 'N/A':
      PBR = None
    elif PBR != 'N/A':
      PBR = float(PBR)
  url = 'https://comp.fnguide.com/SVO2/ASP/SVD_main.asp?pGB=1&gicode=A{}'.format(code)
  res = requests.get(url)
  soup = BeautifulSoup(res.text, 'lxml')
  #EPS, EV/EBITDA, ROE 정보 가져오기
  try:
    table = soup.find_all('table', attrs={'class':'us_table_ty1 h_fix zigbg_no th_topbdno'})[1]
  except:
    continue
  #EPS 정보 가져오기
  EPS = table.find('tbody').find_all('tr')[3].find('td').text
  if ',' in EPS:
    EPS = float(EPS.replace(',',''))
  elif EPS == '\xa0' or EPS == 'N/A':
    EPS = None
  elif len(EPS)>=2:
    EPS = float(EPS)
  #EV/EBITDA 정보 가져오기
  EBITDA = table.find('tbody').find_all('tr')[5].find('td').text
  if ',' in EBITDA:
    EBITDA = float(EBITDA.replace(',',''))
  elif EBITDA == '\xa0' or EBITDA == 'N/A':
    EBITDA = float('-inf')
  elif len(EBITDA) >= 2:
    EBITDA = float(EBITDA)
  #ROE 정보 가져오기
  ROE = table.find('tbody').find_all('tr')[6].find('td').text
  if ',' in ROE:
    ROE = float(ROE.replace(',',''))
  elif ROE == '\xa0' or ROE == 'N/A':
    ROE = None
  elif len(ROE)>=2:
    ROE = float(ROE)
    
  stock_pointer.append([code, PBR, EPS, ROE, EBITDA, upjong_PER])

#가져온 자료 가지고 dataframe 만들기
df_stock_pointer = pd.DataFrame(data = stock_pointer, columns = ['종목코드','PBR','EPS','ROE','EV/EBITDA','업종PER'])
df_stock_pointer = pd.merge(kospi[['시가','종목코드','종목명','중분류','소분류']],df_stock_pointer, how = 'left', on = '종목코드')[:100]
df_stock_pointer.dropna(inplace = True)
df_stock_pointer

,시가,종목코드,종목명,중분류,소분류,PBR,EPS,ROE,EV/EBITDA,업종PER
0,75300.0,005930,삼성전자,반도체와반도체장비,반도체와반도체장비,1.84,3841.0,9.99,8.33,13.91
1,103500.0,000660,SK하이닉스,반도체와반도체장비,반도체와반도체장비,1.25,6532.0,9.53,6.37,13.91
2,398000.0,035420,NAVER,미디어,양방향미디어와서비스,2.52,6097.0,15.21,28.61,7.09
3,919000.0,207940,삼성바이오로직스,제약과생물공학,제약,12.10,3642.0,5.38,125.21,101.06
5,127000.0,035720,카카오,미디어,양방향미디어와서비스,6.49,353.0,2.70,49.25,7.09
...,...,...,...,...,...,...,...,...,...,...
94,308500.0,014680,한솔케미칼,소재,화학,6.24,10864.0,24.50,12.46,15.82
95,70500.0,019170,신풍제약,제약과생물공학,제약,8.12,91.0,1.72,341.11,101.06
97,33950.0,007070,GS리테일,소매,백화점과일반상점,1.07,2187.0,7.49,6.09,42.52
98,77400.0,020150,일진머티리얼즈,기술하드웨어와장비,전자장비와기기,7.77,927.0,7.23,30.42,26.42


In [ ]:
#EV/EBITDA가 나오지 않은 종목들은 모두 작년 EV/EBITDA 정보를 부득이하게 가져왔습니다.
df_stock_pointer.loc[11,'EV/EBITDA'] = 198.72
df_stock_pointer.loc[16,'EV/EBITDA'] = 10.32
df_stock_pointer.loc[21,'EV/EBITDA'] = 20.33
df_stock_pointer.loc[22,'EV/EBITDA'] = 21.53
df_stock_pointer.loc[28,'EV/EBITDA'] = 10.95
df_stock_pointer.loc[31,'EV/EBITDA'] = 17.12
df_stock_pointer.loc[37,'EV/EBITDA'] = 6.08
df_stock_pointer.loc[38,'EV/EBITDA'] = 6.93
df_stock_pointer.loc[43,'EV/EBITDA'] = -207.90
df_stock_pointer.loc[49,'EV/EBITDA'] = 18.61
df_stock_pointer.loc[50,'EV/EBITDA'] = 72.04
df_stock_pointer.loc[58,'EV/EBITDA'] = 63.40
df_stock_pointer.loc[62,'EV/EBITDA'] = 19.71
df_stock_pointer.loc[66,'EV/EBITDA'] = 11.07
df_stock_pointer.loc[68,'EV/EBITDA'] = 16.35
df_stock_pointer.loc[75,'EV/EBITDA'] = 12.61
df_stock_pointer.loc[77,'EV/EBITDA'] = 5.58
df_stock_pointer.loc[79,'EV/EBITDA'] = -63.45
df_stock_pointer.loc[83,'EV/EBITDA'] = -12.73
df_stock_pointer.loc[84,'EV/EBITDA'] = 6.19	
df_stock_pointer.loc[85,'EV/EBITDA'] = 28.05
df_stock_pointer.loc[90,'EV/EBITDA'] = 19.76	
df_stock_pointer.loc[91,'EV/EBITDA'] = 17.99

### 2)크롤링한 자료를 바탕으로 점수 매기기

1. 적정 가격 : `EPS * 업종 PER`로 적정 가격을 책정한 후, 현재 시가가 적정 가격보다 높으면 고평가되어 있는 것이고, 현재 시가가 적정 가격보다 낮으면 저평가 되어 있다고 판단한다. 그래서 시가와 적정가격의 비율을 통해 `시가/적정가격`이 낮을수록 높은 순위를 매긴다.

2. EV/EBITDA : EV/EBITDA가 낮으면 인수 시 투자원금 회수가 빠르고 기업이 저평가되어 있다는 말이므로, 낮을수록 높은 순위를 매김

3. ROE : ROE가 높은 기업은 재무구조상 더 좋다고 판단되기 때문에 ROE가 높은 종목부터 높은 순위를 매긴다.

- 종합 점수 : `(len(df_stock_pointer)*3 - (RANK_평가 + RANK_EV/EBITDA + RANK_ROE)) / 3`를 minmax scaler로 표준화한 점수

In [ ]:
df_stock_pointer['적정가격'] = df_stock_pointer['EPS'] * df_stock_pointer['업종PER'] #EPS * 업종 PER로 적정 가격 책정
df_stock_pointer['시가/적정가격'] = df_stock_pointer['시가']/df_stock_pointer['적정가격'] #시가를 적정 가격으로 나눠서 얼마나 저평가/고평가 되었는지 추정
# PER의 순위 : PER이 낮으면 주가가 기업의 이익에 비해 저평가되었다 하므로 작은 순서부터 순위를 매김
df_stock_pointer['RANK_평가']  = df_stock_pointer['시가/적정가격'].rank(ascending=True)

# EV/EBITDA 순위 : EV/EBITDA가 낮으면 인수 시 투자원금 회수가 빠르고 기업이 저평가되어 있다는 말이므로, 낮을수록 높은 순위를 매김
df_stock_pointer['RANK_EV/EBITDA']  = df_stock_pointer['EV/EBITDA'].rank(ascending=True)

# ROE의 순위 : ROE가 높을수록 가치 있는 주식이라 판단, 순위를 매김
df_stock_pointer['RANK_ROE']  = df_stock_pointer['ROE'].rank(ascending=False)

#종합 점수 매기기
df_stock_pointer['VALUE_SCORE']  = (len(df_stock_pointer) * 3 - (df_stock_pointer['RANK_평가'] + df_stock_pointer['RANK_EV/EBITDA'] + df_stock_pointer['RANK_ROE']))/ 3

#종합 점수 순으로 정렬하기
df_stock_pointer = df_stock_pointer.sort_values(by=['VALUE_SCORE'], ascending= False)
df_stock_pointer.reset_index(drop = True, inplace = True)
df_stock_pointer.head()

,시가,종목코드,종목명,중분류,소분류,PBR,EPS,ROE,EV/EBITDA,업종PER,적정가격,시가/적정가격,RANK_평가,RANK_EV/EBITDA,RANK_ROE,VALUE_SCORE
0,42000.0,137310,에스디바이오센서,건강관리장비와서비스,건강관리장비와용품,2.95,6597.0,139.69,0.01,39.73,262098.81,0.160245,11.0,6.0,1.0,90.000000
1,465000.0,259960,크래프톤,소프트웨어와서비스,게임엔터테인먼트,13.78,13710.0,61.86,0.00,33.32,456817.20,1.017913,37.0,4.5,2.0,81.500000
2,199500.0,011780,금호석유,소재,화학,1.30,17405.0,20.18,5.37,15.82,275347.10,0.724540,25.0,18.0,7.0,79.333333
3,77300.0,021240,코웨이,내구소비재와의류,가정용기기와용품,3.21,5486.0,31.51,4.94,12.26,67258.36,1.149300,44.0,13.0,3.0,76.000000
4,143000.0,066570,LG전자,전자와전기제품,전자제품,1.43,10885.0,13.22,5.15,16.35,177969.75,0.803507,27.0,15.0,18.5,75.833333


In [ ]:
from sklearn.preprocessing import MinMaxScaler

#minmax scaler로 점수 표준화하기
minmax = MinMaxScaler()

minmax.fit(df_stock_pointer[['VALUE_SCORE']])
df_stock_pointer[['VALUE_SCORE']] = minmax.fit_transform(df_stock_pointer[['VALUE_SCORE']])
df_stock_pointer.head()

,시가,종목코드,종목명,중분류,소분류,PBR,EPS,ROE,EV/EBITDA,업종PER,적정가격,시가/적정가격,RANK_평가,RANK_EV/EBITDA,RANK_ROE,VALUE_SCORE
0,42000.0,137310,에스디바이오센서,건강관리장비와서비스,건강관리장비와용품,2.95,6597.0,139.69,0.01,39.73,262098.81,0.160245,11.0,6.0,1.0,1.000000
1,465000.0,259960,크래프톤,소프트웨어와서비스,게임엔터테인먼트,13.78,13710.0,61.86,0.00,33.32,456817.20,1.017913,37.0,4.5,2.0,0.895062
2,199500.0,11780,금호석유,소재,화학,1.30,17405.0,20.18,5.37,15.82,275347.10,0.724540,25.0,18.0,7.0,0.868313
3,77300.0,21240,코웨이,내구소비재와의류,가정용기기와용품,3.21,5486.0,31.51,4.94,12.26,67258.36,1.149300,44.0,13.0,3.0,0.827160
4,143000.0,66570,LG전자,전자와전기제품,전자제품,1.43,10885.0,13.22,5.15,16.35,177969.75,0.803507,27.0,15.0,18.5,0.825103


In [ ]:
df_stock_pointer.to_csv('/content/drive/MyDrive/stock_pointer.csv')